In [2]:
# %pip install vertexai
import vertexai
from vertexai.generative_models import GenerativeModel

vertexai.init(project="talk-to-a-local-427009", location="us-central1")

model = GenerativeModel(model_name="gemini-1.5-flash-001")

response = model.generate_content(
    "What's a good name for a flower shop that specializes in selling bouquets of dried flowers?"
)

print(response.text)

Here are some names for a flower shop specializing in dried flowers, playing on different themes:

**Simple & Elegant:**

* Bloom & Preserve
* The Dried Bouquet
* Everlasting Blooms
* Timeless Flowers
* The Bloom Archive

**Whimsical & Romantic:**

* Whispers of Wildflowers
* Sun-Kissed Blooms
* The Paper Petal
* Dried Dreams
* Forever in Bloom

**Modern & Minimalist:**

* Bloom & Co.
* Botanical & Co.
* The Dried Flower Bar
* Bloom Lab
* The Wild Bloom

**Location-Specific:**

* [Your City] Dried Blooms
* [Your Street] Flower Co.
* [Local Landmark] Blooms

**Other Ideas:**

* The Herbarium
* The Botanical Studio
* The Dried Flower Gallery
* The Flower Alchemist
* The Bloom Preserve

**Tips for Choosing a Name:**

* **Consider your target audience:** Who are you trying to attract?
* **Keep it memorable and easy to pronounce:**  A catchy name will stick in people's minds.
* **Make sure the name is available:** Check if the domain name and social media handles are available.
* **Reflect 

In [2]:
from openai import AsyncOpenAI
import json

keys = json.load(open("secrets.json"))

client = AsyncOpenAI(
    base_url="http://101.35.52.226:9090/v1",
    api_key=keys['api_key'],
    timeout=45,
)


async def chat(prompt, stream=False, temperature=0.0, n=1):
    response = await client.chat.completions.create(
        model="qwen-110b-chat",
        messages=[{"role": "user", "content": prompt}],
        stream=stream,
        max_tokens=512,
        temperature=temperature,
        n=n,
        stop=["<|endoftext|>", "<|im_end|>"],
    )
    if not stream:
        if n == 1:
            return response.choices[0].message.content.strip()
        return response.choices
    return response